In [18]:
import pandas as pd
import json
import numpy as np

Данные полученные через [DBPedia](https://dbpedia.org/sparql).
Все победы Михаэля Шумахера. Гран при по годам и кто занял второе место:

SPARQL - скрипт: 
```
select distinct ?s ?nameLabel ?year ?secondLabel
where {
    ?s rdf:type dbo:GrandPrix; dbo:firstDriver dbr:Michael_Schumacher .
    ?s dbp:nameOfRace ?name .
    ?s dbp:year ?year .
    ?s dbo:secondDriver ?second .
    ?second rdfs:label ?secondLabel .
    FILTER (lang(?secondLabel) = 'en')
    ?name rdfs:label ?nameLabel .
    FILTER (lang(?nameLabel) = 'en')
} 
ORDER BY DESC(?year)
```

In [23]:
data1 = np.empty([0,4])
with open('data.json') as f:
    d = json.load(f)
    # data1 = np.array(['uri', 'gp label', 'year', 'second driver'], ndmin=2,dtype='<U50')
    for line in d['results']['bindings']:
        t = np.array([line['s']['value'],line['nameLabel']['value'],line['year']['value'],line['secondLabel']['value']])
        data1 = np.vstack([data1, t])

data = np.vstack([np.array(['uri', 'gp label', 'year', 'second driver'], ndmin=2,dtype='<U50'), data1])
df = pd.DataFrame(data=data[1:,1:],
              index=data[1:,0],
              columns=data[0,1:])
    
df

,gp label,year,second driver
http://dbpedia.org/resource/2006_French_Grand_Prix,French Grand Prix,2006,Fernando Alonso
http://dbpedia.org/resource/2006_United_States_Grand_Prix,United States Grand Prix,2006,Felipe Massa
http://dbpedia.org/resource/2006_European_Grand_Prix,European Grand Prix,2006,Fernando Alonso
http://dbpedia.org/resource/2006_San_Marino_Grand_Prix,San Marino Grand Prix,2006,Fernando Alonso
http://dbpedia.org/resource/2006_Chinese_Grand_Prix,Chinese Grand Prix,2006,Fernando Alonso
...,...,...,...
http://dbpedia.org/resource/1994_European_Grand_Prix,European Grand Prix,1994,Damon Hill
http://dbpedia.org/resource/1994_Hungarian_Grand_Prix,Hungarian Grand Prix,1994,Damon Hill
http://dbpedia.org/resource/1994_Pacific_Grand_Prix,Pacific Grand Prix,1994,Gerhard Berger
http://dbpedia.org/resource/1993_Portuguese_Grand_Prix,Portuguese Grand Prix,1993,Alain Prost


Парсил данные с основной страницы Михаеля в википедии - оттуда вынул ссылки на сезоны в которых он соревновался, затем парсил отдельно каждый сезон. 
Скрипт: `parse.py`
    Результат работы скрипта сохраняю в `processed_data.json`

Принцип оценки качества работы скрипта был выбран такой:
Если неправильно оценена победа в гонке 0 баллов
Если победа найдена правильно но не найден кто занял второе место или его имя неверно - 0.5 баллов
Если правильно определена и побела и кто занял 2е место - 1 балл
Считается разность баллов

В первых версиях скрипта неправильно анотировались некоторые гонки, например 
`The 69-lap race was won by Williams driver Ralf Schumacher after starting from the second position`
система путала Шумахера с братом или
`This race, Michael Schumacher's first Ferrari victory, is generally regarded as one of his finest.`
на основе этих и других примеров были написаны юнит тесты для того чтоб более безопасно и быстро итерировать новые идеи.

Так же пришлось поработать с false-positives как, например, 
`The race is remembered for an incident involving the two title contenders Damon Hill and Michael Schumacher which forced both to retire and resulted in Schumacher winning the World Drivers' Championship.`
где системa сначала думала что Шумахер выиграл конкретную гонку иди
`Michael Schumacher won the 37th pole position of his career by setting the fastest lap in qualifying` где система путала квалификацию и гонку

In [52]:
with open('processed_data.json') as f:
    res = json.load(f)
    # data2 = np.array(['gp label', 'year', 'second driver'], ndmin=2,dtype='<U50')
    data2 = np.empty([0,4])
    for t in res:
        tmp = np.array(t)
        data2 = np.vstack([data2, tmp])


gp1 = data1[:, 0]
gp2 = data2[:, 0]

wrong_second_place=[]

_, x_ind, y_ind = np.intersect1d(gp1, gp2, return_indices=True)
score = 0
for dp, p in zip(x_ind, y_ind):
    #if processed is same as db - give 1 point
    #if not give - 0,5 point
    if dp==p:
        score+=1
    else:
        wrong_second_place.append(np.append(data2[p], data1[p][3]))
        score+=0.5

print("Accuracy: {:.2f}%".format(score*100/len(gp1)))

Accuracy: 25.82%


#### Гонки где не была определена победа:

In [55]:
a = np.setdiff1d(gp1, gp2, assume_unique=True)
len(pd.DataFrame(data=a,columns=['uri']))

44

#### Гонки где неправильно был определён гонщик на 2м месте:

In [49]:
pd.DataFrame(data=wrong_second_place,columns=['uri','gp label', 'year', 'parsed second driver', 'true second driver'])

,uri,gp label,year,parsed second driver,true second driver
0,http://dbpedia.org/resource/1992_Belgian_Grand...,Belgian Grand Prix,1992,Champion Nigel Mansell,Fernando Alonso
1,http://dbpedia.org/resource/1993_Portuguese_Gr...,Portuguese Grand Prix,1993,Alain Prost,Felipe Massa
2,http://dbpedia.org/resource/1994_Brazilian_Gra...,Brazilian Grand Prix,1994,Damon Hill,Fernando Alonso
3,http://dbpedia.org/resource/1994_Canadian_Gran...,Canadian Grand Prix,1994,Damon Hill,Fernando Alonso
4,http://dbpedia.org/resource/1994_European_Gran...,European Grand Prix,1994,None,Fernando Alonso
5,http://dbpedia.org/resource/1994_French_Grand_...,French Grand Prix,1994,Damon Hill,Felipe Massa
6,http://dbpedia.org/resource/1994_Hungarian_Gra...,Hungarian Grand Prix,1994,Damon Hill,Kimi Räikkönen
7,http://dbpedia.org/resource/1994_Monaco_Grand_...,Monaco Grand Prix,1994,Martin Brundle,Rubens Barrichello
8,http://dbpedia.org/resource/1994_Pacific_Grand...,Pacific Grand Prix,1994,Gerhard Berger,Rubens Barrichello
9,http://dbpedia.org/resource/1994_San_Marino_Gr...,San Marino Grand Prix,1994,Nicola Larini,Rubens Barrichello
